In [1]:
import pandas as pd
import numpy as np
import torch
import json
import os
from Network.deepnetwork2 import ImitationNetwork
from Memory.dataset import *
from torch.utils.data import DataLoader
from Utils.utils import *
from sklearn.metrics import precision_score, recall_score
import torch.nn as nn

In [2]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('TensorBoard/1stImitationAgent_AllActions_NewData_AllReward')


In [ ]:
len(decode_state_old_test(a))

In [ ]:
a = {'player_id': 1, 'player_life': 39, 'player_gold': 0, 'opponent_life': 39, 'time': 59.839996337890625, 'player_board_card_info': [{'id': 23, 'status': 1, 'damage': 1, 'life': 0}, {'id': 7, 'status': 2, 'damage': 1, 'life': 2}, {'id': 50, 'status': 2, 'damage': 2, 'life': 1}, {'id': 50, 'status': 2, 'damage': 
2, 'life': 1}, {'id': 31, 'status': 2, 'damage': 1, 'life': 1}, {'id': 20, 'status': 0, 'damage': 3, 'life': 3}, {'id': 0, 'status': 0, 'damage': 0, 'life': 0}], 'opponent_board_card_info': [{'id': 23, 'status': 0, 'damage': 1, 'life': 2}, {'id': 31, 'status': 0, 'damage': 1, 'life': 0}, {'id': 7, 'status': 2, 'damage': 1, 'life': 2}, {'id': 34, 'status': 0, 'damage': 2, 'life': 1}, {'id': 0, 'status': 0, 'damage': 0, 'life': 0}, {'id': 0, 'status': 0, 'damage': 0, 'life': 0}, {'id': 0, 'status': 0, 'damage': 0, 'life': 0}], 'player_hand_card_id': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'player_remaining_cards_on_deck': [1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1], 'opponent_remaining_cards': [0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 
1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'player_used_cards': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 
0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'opponent_used_cards': [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],'action':21}

In [3]:
dataset = Feeder1('Log/ImitationLog/Train')

In [4]:
dataloader = DataLoader(dataset,batch_size = 256, num_workers = 20,shuffle=True)

In [5]:
def accuracy(pred, label):
    pred = torch.argmax(pred, dim=1).long()
#     print(pred)
    label = torch.argmax(label,dim=1).long()
    acc = torch.mean((pred == label).float())
    pred = pred.detach().cpu().numpy()
    label = label.detach().cpu().numpy()
    p = precision_score(label, pred,average='micro')
    r = recall_score(label, pred,average='micro')
    return p,r,acc 

class AverageMeter(object):

    def __init__(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [6]:
model = ImitationNetwork(0.1,290,'Test',740,'Test')
model.to(torch.device('cuda:1'))

ImitationNetwork(
  (dense1): Linear(in_features=740, out_features=2048, bias=True)
  (dense2): Linear(in_features=2048, out_features=1024, bias=True)
  (dense3): Linear(in_features=1024, out_features=512, bias=True)
  (dense4): Linear(in_features=512, out_features=256, bias=True)
  (dense5): Linear(in_features=256, out_features=128, bias=True)
  (dense6): Linear(in_features=128, out_features=64, bias=True)
  (dense7): Linear(in_features=64, out_features=32, bias=True)
  (dense8): Linear(in_features=32, out_features=290, bias=True)
  (loss): CrossEntropyLoss()
)

In [7]:
crit = nn.BCEWithLogitsLoss().to(torch.device('cuda:1'))
losses = 0.0
accs  = 0.0
precisions  = AverageMeter()
recalls  = AverageMeter()

opt = torch.optim.SGD(model.parameters(), 0.01, 
                          momentum=0.9, 
                          weight_decay=1e-4) 
# checkpoint = torch.load('Models/1stImitationAgent_OldData.ckpt')
# model.load_state_dict(checkpoint)
for epoch in range(20):
    for i, (data,reward,label) in enumerate(dataloader):
        data_,reward_,label_ = map(lambda x:x.to(torch.device('cuda:1')),(data,reward,label))

        pred = model.forward(data_.float())
#         y_onehot = label.detach().numpy()
#         y_onehot = (np.arange(56) == y_onehot[:,None]).astype(np.float32)
#         y_onehot = torch.from_numpy(y_onehot).to(torch.device('cuda:1'))
        loss = crit(pred,torch.nn.functional.softmax(reward_))
        p,r, acc = accuracy(pred, torch.nn.functional.softmax(reward_))

        opt.zero_grad()
        loss.backward()
        opt.step()

#         losses.update(loss.item(),530)
#         accs.update(acc.item(),530)
        losses += loss.item()
        accs += acc.item()
        precisions.update(p, 530)
        recalls.update(r,530)

        if i % 5 == 0:
            print('Epoch:[{0}][{1}/{2}]\t'
#                   'Loss {losses.val:.3f} ({losses.avg:.3f})\t'
#                   'Accuracy {accs.val:.3f} ({accs.avg:.3f})\t'
                  'Loss {losses:.3f} \t'
                  'Accuracy {accs:.3f}\t'
                  'Precison {precisions.val:.3f} ({precisions.avg:.3f})\t'
                  'Recall {recalls.val:.3f} ({recalls.avg:.3f})'.format(
                        epoch,i,len(dataloader) ,losses=loss, accs=acc, 
                        precisions=precisions, recalls=recalls))
        if i % 10 == 0:
            torch.save(model.state_dict(), 'Models/ImitationAgent_NewData_AllReward.ckpt')
            # ...log the running loss
            writer.add_scalar('training loss',
                            losses / 10,
                            epoch * len(dataloader) + i)

            # ...log a Matplotlib Figure showing the model's predictions on a
            # random mini-batch
            writer.add_scalar('accuracy',
                            accs / 10,
                            global_step=epoch * len(dataloader) + i)
            losses = 0.0
            accs = 0.0
    #         print('Epoch:[{0}][{1}/{2}]\t'
    #               'Loss {losses:.3f} \t'
    #               'Accuracy {accs:.3f}\t'
    #               'Precison {precisions:.3f}\t'
    #               'Recall {recalls:.3f}'.format(
    #                     epoch, losses=loss, accs=acc, 
    #                     precisions=p, recalls=r))
    

/home/kietcdx/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/kietcdx/myenv/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch:[0][0/273]	Loss 0.695 	Accuracy 0.000	Precison 0.000 (0.000)	Recall 0.000 (0.000)
Epoch:[0][5/273]	Loss 0.695 	Accuracy 0.000	Precison 0.000 (0.000)	Recall 0.000 (0.000)
Epoch:[0][10/273]	Loss 0.695 	Accuracy 0.000	Precison 0.000 (0.000)	Recall 0.000 (0.000)
Epoch:[0][15/273]	Loss 0.694 	Accuracy 0.000	Precison 0.000 (0.000)	Recall 0.000 (0.000)
Epoch:[0][20/273]	Loss 0.694 	Accuracy 0.000	Precison 0.000 (0.000)	Recall 0.000 (0.000)
Epoch:[0][25/273]	Loss 0.693 	Accuracy 0.000	Precison 0.000 (0.000)	Recall 0.000 (0.000)
Epoch:[0][30/273]	Loss 0.693 	Accuracy 0.000	Precison 0.000 (0.000)	Recall 0.000 (0.000)
Epoch:[0][35/273]	Loss 0.692 	Accuracy 0.000	Precison 0.000 (0.000)	Recall 0.000 (0.000)
Epoch:[0][40/273]	Loss 0.692 	Accuracy 0.000	Precison 0.000 (0.000)	Recall 0.000 (0.000)
Epoch:[0][45/273]	Loss 0.691 	Accuracy 0.000	Precison 0.000 (0.000)	Recall 0.000 (0.000)
Epoch:[0][50/273]	Loss 0.691 	Accuracy 0.000	Precison 0.000 (0.000)	Recall 0.000 (0.000)
Epoch:[0][55/273]	Loss 

Epoch:[1][185/273]	Loss 0.031 	Accuracy 0.082	Precison 0.082 (0.000)	Recall 0.082 (0.000)
Epoch:[1][190/273]	Loss 0.032 	Accuracy 0.066	Precison 0.066 (0.001)	Recall 0.066 (0.001)
Epoch:[1][195/273]	Loss 0.033 	Accuracy 0.023	Precison 0.023 (0.002)	Recall 0.023 (0.002)
Epoch:[1][200/273]	Loss 0.032 	Accuracy 0.047	Precison 0.047 (0.002)	Recall 0.047 (0.002)
Epoch:[1][205/273]	Loss 0.030 	Accuracy 0.160	Precison 0.160 (0.003)	Recall 0.160 (0.003)
Epoch:[1][210/273]	Loss 0.029 	Accuracy 0.141	Precison 0.141 (0.004)	Recall 0.141 (0.004)
Epoch:[1][215/273]	Loss 0.028 	Accuracy 0.172	Precison 0.172 (0.006)	Recall 0.172 (0.006)
Epoch:[1][220/273]	Loss 0.026 	Accuracy 0.156	Precison 0.156 (0.007)	Recall 0.156 (0.007)
Epoch:[1][225/273]	Loss 0.026 	Accuracy 0.184	Precison 0.184 (0.009)	Recall 0.184 (0.009)
Epoch:[1][230/273]	Loss 0.026 	Accuracy 0.188	Precison 0.188 (0.010)	Recall 0.188 (0.010)
Epoch:[1][235/273]	Loss 0.027 	Accuracy 0.102	Precison 0.102 (0.011)	Recall 0.102 (0.011)
Epoch:[1][

Epoch:[3][105/273]	Loss 0.022 	Accuracy 0.215	Precison 0.215 (0.073)	Recall 0.215 (0.073)
Epoch:[3][110/273]	Loss 0.024 	Accuracy 0.129	Precison 0.129 (0.074)	Recall 0.129 (0.074)
Epoch:[3][115/273]	Loss 0.024 	Accuracy 0.125	Precison 0.125 (0.074)	Recall 0.125 (0.074)
Epoch:[3][120/273]	Loss 0.023 	Accuracy 0.180	Precison 0.180 (0.075)	Recall 0.180 (0.075)
Epoch:[3][125/273]	Loss 0.023 	Accuracy 0.121	Precison 0.121 (0.075)	Recall 0.121 (0.075)
Epoch:[3][130/273]	Loss 0.024 	Accuracy 0.137	Precison 0.137 (0.075)	Recall 0.137 (0.075)
Epoch:[3][135/273]	Loss 0.023 	Accuracy 0.148	Precison 0.148 (0.076)	Recall 0.148 (0.076)
Epoch:[3][140/273]	Loss 0.024 	Accuracy 0.145	Precison 0.145 (0.076)	Recall 0.145 (0.076)
Epoch:[3][145/273]	Loss 0.024 	Accuracy 0.129	Precison 0.129 (0.076)	Recall 0.129 (0.076)
Epoch:[3][150/273]	Loss 0.023 	Accuracy 0.164	Precison 0.164 (0.077)	Recall 0.164 (0.077)
Epoch:[3][155/273]	Loss 0.023 	Accuracy 0.137	Precison 0.137 (0.077)	Recall 0.137 (0.077)
Epoch:[3][

Epoch:[5][15/273]	Loss 0.023 	Accuracy 0.141	Precison 0.141 (0.098)	Recall 0.141 (0.098)
Epoch:[5][20/273]	Loss 0.023 	Accuracy 0.105	Precison 0.105 (0.098)	Recall 0.105 (0.098)
Epoch:[5][25/273]	Loss 0.022 	Accuracy 0.168	Precison 0.168 (0.099)	Recall 0.168 (0.099)
Epoch:[5][30/273]	Loss 0.023 	Accuracy 0.113	Precison 0.113 (0.099)	Recall 0.113 (0.099)
Epoch:[5][35/273]	Loss 0.022 	Accuracy 0.145	Precison 0.145 (0.099)	Recall 0.145 (0.099)
Epoch:[5][40/273]	Loss 0.023 	Accuracy 0.109	Precison 0.109 (0.099)	Recall 0.109 (0.099)
Epoch:[5][45/273]	Loss 0.023 	Accuracy 0.141	Precison 0.141 (0.099)	Recall 0.141 (0.099)
Epoch:[5][50/273]	Loss 0.022 	Accuracy 0.152	Precison 0.152 (0.099)	Recall 0.152 (0.099)
Epoch:[5][55/273]	Loss 0.023 	Accuracy 0.172	Precison 0.172 (0.100)	Recall 0.172 (0.100)
Epoch:[5][60/273]	Loss 0.023 	Accuracy 0.137	Precison 0.137 (0.100)	Recall 0.137 (0.100)
Epoch:[5][65/273]	Loss 0.022 	Accuracy 0.129	Precison 0.129 (0.100)	Recall 0.129 (0.100)
Epoch:[5][70/273]	Los

...

Epoch:[11][210/273]	Loss 0.021 	Accuracy 0.195	Precison 0.195 (0.127)	Recall 0.195 (0.127)
Epoch:[11][215/273]	Loss 0.022 	Accuracy 0.156	Precison 0.156 (0.127)	Recall 0.156 (0.127)
Epoch:[11][220/273]	Loss 0.022 	Accuracy 0.125	Precison 0.125 (0.127)	Recall 0.125 (0.127)
Epoch:[11][225/273]	Loss 0.022 	Accuracy 0.168	Precison 0.168 (0.127)	Recall 0.168 (0.127)
Epoch:[11][230/273]	Loss 0.022 	Accuracy 0.129	Precison 0.129 (0.127)	Recall 0.129 (0.127)
Epoch:[11][235/273]	Loss 0.022 	Accuracy 0.145	Precison 0.145 (0.127)	Recall 0.145 (0.127)
Epoch:[11][240/273]	Loss 0.022 	Accuracy 0.141	Precison 0.141 (0.127)	Recall 0.141 (0.127)
Epoch:[11][245/273]	Loss 0.022 	Accuracy 0.156	Precison 0.156 (0.127)	Recall 0.156 (0.127)
Epoch:[11][250/273]	Loss 0.022 	Accuracy 0.133	Precison 0.133 (0.127)	Recall 0.133 (0.127)
Epoch:[11][255/273]	Loss 0.022 	Accuracy 0.180	Precison 0.180 (0.128)	Recall 0.180 (0.128)
Epoch:[11][260/273]	Loss 0.022 	Accuracy 0.164	Precison 0.164 (0.128)	Recall 0.164 (0.128)

Epoch:[13][125/273]	Loss 0.022 	Accuracy 0.188	Precison 0.188 (0.130)	Recall 0.188 (0.130)
Epoch:[13][130/273]	Loss 0.022 	Accuracy 0.156	Precison 0.156 (0.130)	Recall 0.156 (0.130)
Epoch:[13][135/273]	Loss 0.021 	Accuracy 0.156	Precison 0.156 (0.130)	Recall 0.156 (0.130)
Epoch:[13][140/273]	Loss 0.022 	Accuracy 0.172	Precison 0.172 (0.130)	Recall 0.172 (0.130)
Epoch:[13][145/273]	Loss 0.022 	Accuracy 0.105	Precison 0.105 (0.130)	Recall 0.105 (0.130)
Epoch:[13][150/273]	Loss 0.022 	Accuracy 0.121	Precison 0.121 (0.130)	Recall 0.121 (0.130)
Epoch:[13][155/273]	Loss 0.022 	Accuracy 0.129	Precison 0.129 (0.130)	Recall 0.129 (0.130)
Epoch:[13][160/273]	Loss 0.022 	Accuracy 0.148	Precison 0.148 (0.130)	Recall 0.148 (0.130)
Epoch:[13][165/273]	Loss 0.022 	Accuracy 0.125	Precison 0.125 (0.130)	Recall 0.125 (0.130)
Epoch:[13][170/273]	Loss 0.022 	Accuracy 0.164	Precison 0.164 (0.130)	Recall 0.164 (0.130)
Epoch:[13][175/273]	Loss 0.022 	Accuracy 0.156	Precison 0.156 (0.130)	Recall 0.156 (0.130)

Epoch:[15][40/273]	Loss 0.021 	Accuracy 0.176	Precison 0.176 (0.132)	Recall 0.176 (0.132)
Epoch:[15][45/273]	Loss 0.022 	Accuracy 0.145	Precison 0.145 (0.132)	Recall 0.145 (0.132)
Epoch:[15][50/273]	Loss 0.022 	Accuracy 0.148	Precison 0.148 (0.132)	Recall 0.148 (0.132)
Epoch:[15][55/273]	Loss 0.022 	Accuracy 0.148	Precison 0.148 (0.132)	Recall 0.148 (0.132)
Epoch:[15][60/273]	Loss 0.021 	Accuracy 0.180	Precison 0.180 (0.132)	Recall 0.180 (0.132)
Epoch:[15][65/273]	Loss 0.022 	Accuracy 0.148	Precison 0.148 (0.132)	Recall 0.148 (0.132)
Epoch:[15][70/273]	Loss 0.022 	Accuracy 0.148	Precison 0.148 (0.132)	Recall 0.148 (0.132)
Epoch:[15][75/273]	Loss 0.022 	Accuracy 0.152	Precison 0.152 (0.132)	Recall 0.152 (0.132)
Epoch:[15][80/273]	Loss 0.022 	Accuracy 0.180	Precison 0.180 (0.132)	Recall 0.180 (0.132)
Epoch:[15][85/273]	Loss 0.021 	Accuracy 0.160	Precison 0.160 (0.132)	Recall 0.160 (0.132)
Epoch:[15][90/273]	Loss 0.022 	Accuracy 0.152	Precison 0.152 (0.132)	Recall 0.152 (0.132)
Epoch:[15]

Epoch:[16][230/273]	Loss 0.021 	Accuracy 0.207	Precison 0.207 (0.134)	Recall 0.207 (0.134)
Epoch:[16][235/273]	Loss 0.022 	Accuracy 0.121	Precison 0.121 (0.134)	Recall 0.121 (0.134)
Epoch:[16][240/273]	Loss 0.022 	Accuracy 0.129	Precison 0.129 (0.134)	Recall 0.129 (0.134)
Epoch:[16][245/273]	Loss 0.021 	Accuracy 0.199	Precison 0.199 (0.134)	Recall 0.199 (0.134)
Epoch:[16][250/273]	Loss 0.021 	Accuracy 0.145	Precison 0.145 (0.134)	Recall 0.145 (0.134)
Epoch:[16][255/273]	Loss 0.022 	Accuracy 0.152	Precison 0.152 (0.134)	Recall 0.152 (0.134)
Epoch:[16][260/273]	Loss 0.021 	Accuracy 0.207	Precison 0.207 (0.134)	Recall 0.207 (0.134)
Epoch:[16][265/273]	Loss 0.021 	Accuracy 0.152	Precison 0.152 (0.134)	Recall 0.152 (0.134)
Epoch:[16][270/273]	Loss 0.022 	Accuracy 0.160	Precison 0.160 (0.134)	Recall 0.160 (0.134)
Epoch:[17][0/273]	Loss 0.022 	Accuracy 0.117	Precison 0.117 (0.134)	Recall 0.117 (0.134)
Epoch:[17][5/273]	Loss 0.022 	Accuracy 0.172	Precison 0.172 (0.134)	Recall 0.172 (0.134)
Epo

Epoch:[18][145/273]	Loss 0.022 	Accuracy 0.105	Precison 0.105 (0.135)	Recall 0.105 (0.135)
Epoch:[18][150/273]	Loss 0.021 	Accuracy 0.129	Precison 0.129 (0.135)	Recall 0.129 (0.135)
Epoch:[18][155/273]	Loss 0.021 	Accuracy 0.148	Precison 0.148 (0.135)	Recall 0.148 (0.135)
Epoch:[18][160/273]	Loss 0.022 	Accuracy 0.160	Precison 0.160 (0.135)	Recall 0.160 (0.135)
Epoch:[18][165/273]	Loss 0.022 	Accuracy 0.133	Precison 0.133 (0.135)	Recall 0.133 (0.135)
Epoch:[18][170/273]	Loss 0.021 	Accuracy 0.180	Precison 0.180 (0.135)	Recall 0.180 (0.135)
Epoch:[18][175/273]	Loss 0.021 	Accuracy 0.164	Precison 0.164 (0.135)	Recall 0.164 (0.135)
Epoch:[18][180/273]	Loss 0.021 	Accuracy 0.141	Precison 0.141 (0.135)	Recall 0.141 (0.135)
Epoch:[18][185/273]	Loss 0.022 	Accuracy 0.148	Precison 0.148 (0.135)	Recall 0.148 (0.135)
Epoch:[18][190/273]	Loss 0.022 	Accuracy 0.137	Precison 0.137 (0.135)	Recall 0.137 (0.135)
Epoch:[18][195/273]	Loss 0.022 	Accuracy 0.133	Precison 0.133 (0.135)	Recall 0.133 (0.135)

In [ ]:
dataset_test = Feeder3('Log/ImitationLog/Test/')
dataloader1 = DataLoader(dataset_test,batch_size = 256, num_workers = 20,shuffle=True)

In [ ]:
crit = nn.BCEWithLogitsLoss().to(torch.device('cuda:1'))
losses = AverageMeter()
accs  = AverageMeter()
precisions  = AverageMeter()
recalls  = AverageMeter()

opt = torch.optim.SGD(model.parameters(), 0.01, 
                          momentum=0.9, weight_decay=1e-4) 
# checkpoint = torch.load('Models/1stImitationAgent_OldData.ckpt')
# model.load_state_dict(checkpoint)
epoch = 0
for i, (data,reward,label) in enumerate(dataloader1):
    data_,reward_,label_ = map(lambda x:x.to(torch.device('cuda:1')),(data,reward,label))

    pred = model.forward(data_.float())
#         y_onehot = label.detach().numpy()
#         y_onehot = (np.arange(56) == y_onehot[:,None]).astype(np.float32)
#         y_onehot = torch.from_numpy(y_onehot).to(torch.device('cuda:1'))
    loss = crit(pred,torch.nn.functional.softmax(reward_))
    p,r, acc = accuracy(pred, torch.nn.functional.softmax(reward_))

#         opt.zero_grad()
#         loss.backward()
#         opt.step()

    losses.update(loss.item(),530)
    accs.update(acc.item(),530)
#     losses += loss.item()
#     accs += acc.item()
    
    precisions.update(p, 530)
    recalls.update(r,530)
    print('Epoch:[{0}][{1}/{2}]\t'
          'Loss {losses.val:.3f} ({losses.avg:.3f})\t'
          'Accuracy {accs.val:.3f} ({accs.avg:.3f})\t'
#           'Loss {losses:.3f} \t'
#           'Accuracy {accs:.3f}\t'
          'Precison {precisions.val:.3f} ({precisions.avg:.3f})\t'
          'Recall {recalls.val:.3f} ({recalls.avg:.3f})'.format(
                epoch,i,len(dataloader1) ,losses=losses, accs=accs, 
                        precisions=precisions, recalls=recalls))

In [ ]:
accs

In [ ]:
torch.save(model.state_dict(), 'Models/1stImitationAgent_OldData.ckpt')
# checkpoint = torch.load('Models/1stImitationAgent_OldData.ckpt')
# model.load_state_dict(checkpoint)

In [ ]:
max([1,2,3])

In [ ]:
input

In [ ]:
pred[0]

In [ ]:
f = open('Log/2021-03-11_23-01-35_1.json')
data = json.load(f)

In [ ]:
data = data['logs']

In [ ]:
IMITATION_LOG_DIR = 'Log/'

In [ ]:
a = list()
for i in os.listdir(IMITATION_LOG_DIR):
    f = open(IMITATION_LOG_DIR + i)
    try:
        a.extend(json.load(f)['logs'])
    except json.JSONDecodeError:
        f.close()
        continue
    f.close()

In [ ]:
player_life = [str_env['player_life']]
player_gold = [str_env['player_life']] 
opponent_life = [str_env['opponent_life']]
time = [str_env['time']]
player_board_card_info = list()
opponent_board_card_info = list()

for x in str_env['player_board_card_info']:
    player_board_card_info.append(x['id'])
    player_board_card_info.append(x['status'])
    player_board_card_info.append(x['damage'])
    player_board_card_info.append(x['life'])

for x in str_env['opponent_board_card_info']:
    opponent_board_card_info.append(x['id'])
    opponent_board_card_info.append(x['status'])
    opponent_board_card_info.append(x['damage'])
    opponent_board_card_info.append(x['life'])
    
player_hand_card_id = str_env['player_hand_card_id']

In [ ]:
state = player_life + player_gold + opponent_life + time + player_board_card_info + opponent_board_card_info + player_hand_card_id

In [ ]:
len(state)

In [ ]:
a = torch.Tensor([1,2,2,2])
b = torch.Tensor([1,3,55,3])

In [ ]:
torch.add(a,b)

In [ ]:
test == None

In [ ]:
np.random.choice(1)

In [ ]:
info = pd.read_csv('CardIDList.csv')

In [ ]:
card_info = info[info['ID']==2]

In [ ]:
card_info

In [ ]:
info.at[56,"Name"]